In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

from main_script import download_images_for_points
from process_data import get_gvi_per_buffer
from osmnx_road_network import select_points_on_road_network, get_features_on_points, get_road_network_with_points, select_points_within_buffers
import geopandas as gpd
from datetime import timedelta
from time import time

In [2]:
file = "Test_multiple_home_locations.gpkg" # File with address points
layer = "Test_multiple_home_locations"
buffer_distance = 500 # buffer distance in meters
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf" # Access token for mapillary

In [3]:
# Read the file as a GeoDataFrame
gdf_points = gpd.read_file(file, layer=layer) # The crs must use meters

gdf_points['buffer'] = gdf_points['geometry'].buffer(buffer_distance)

In [4]:
# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf_points.to_crs(epsg=4326, inplace=True)

# Reproject the 'buffer' column to EPSG 4326
gdf_points['buffer'] = gdf_points['buffer'].to_crs(epsg=4326)

In [5]:
road = get_road_network_with_points(gdf_points['buffer'])

# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
road.to_crs(epsg=4326, inplace=True)

In [6]:
road_points = select_points_on_road_network(road)

In [7]:
points = select_points_within_buffers(gdf_points, road_points)

In [9]:
# Get features for each point
features = get_features_on_points(points, access_token)

file_path = os.path.join("results", "points.gpkg")
features.to_file(file_path, driver="GPKG")

# Get the initial time
start_time = time()
    
gvi_per_point = download_images_for_points(features, access_token)
# Get the final time
end_time = time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Format the elapsed time as "hh:mm:ss"
formatted_time = str(timedelta(seconds=elapsed_time))

print(f"Running time: {formatted_time}")

Running time: 0:27:44.493010


In [54]:
def get_gvi_per_buffer(buffered_points, gvi_per_point):
    joined = gpd.sjoin(gvi_per_point, buffered_points.set_geometry('buffer'), how='inner', predicate='within')

    # Group the points by buffer
    grouped = joined.groupby('index_right')

    # Convert 'grouped' to a DataFrame
    grouped_df = grouped.apply(lambda x: x.reset_index(drop=True))
    grouped_df = grouped_df[["geometry_left", "GVI", "is_panoramic", "missing"]].reset_index()
    # Convert grouped_df to a GeoDataFrame
    grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry_left')

    # Calculate the average 'gvi' for each group
    avg_gvi = grouped['GVI'].mean().reset_index()
    point_count = grouped['GVI'].count().reset_index(name='Point_Count')

    # Merge with the buffered_points dataframe to get the buffer geometries
    result = avg_gvi.merge(buffered_points, left_on='index_right', right_index=True)
    result = result.merge(point_count, on='index_right')
    # Convert the result to a GeoDataFrame
    result = gpd.GeoDataFrame(result[['geometry', 'GVI', 'Point_Count']])

    return result, grouped_gdf

In [55]:
gvi_buffer, points = get_gvi_per_buffer(gdf_points, gvi_per_point)

In [56]:
display(gvi_buffer)

,geometry,GVI,Point_Count
0,POINT (-2.17238 53.43243),0.154665,36
1,POINT (-2.21250 53.44085),0.274186,120
2,POINT (-2.17267 53.45455),0.127187,99


In [57]:
display(points)

# index_right has the UID of the point (or buffer)
# geometry_left has the coordinates of the sample point that is within the buffer
# GVI has the GVI value, NaN in case the image is missing
# is_panoramic indicates if the image is panoramic, None in case it is missing
# missing indicates if the images was usable (road centers were identified) or not

,index_right,level_1,geometry_left,GVI,is_panoramic,missing
0,0,0,POINT (-2.17795 53.42943),0.093304,False,False
1,0,1,POINT (-2.16738 53.43337),0.239231,False,False
2,0,2,POINT (-2.16857 53.43363),0.239231,False,False
3,0,3,POINT (-2.16753 53.43313),NaN,None,True
4,0,4,POINT (-2.16714 53.43382),0.239231,False,False
...,...,...,...,...,...,...
342,2,122,POINT (-2.17715 53.45728),0.016838,False,False
343,2,123,POINT (-2.17672 53.45603),0.020062,False,False
344,2,124,POINT (-2.17591 53.45395),0.014713,False,False
345,2,125,POINT (-2.17808 53.45289),0.000000,False,False


In [58]:
file_path = os.path.join("results", "GVI-points.gpkg")
points.to_file(file_path, driver="GPKG")

file_path = os.path.join("results", "GVI-buffer.gpkg")
gvi_buffer.to_file(file_path, driver="GPKG")